In [2]:
#s数据准备和各种库
import tensorflow.compat.v1  as tf
import matplotlib.pyplot as plt #载入matplot库
###加这个，避免画图不显示
%matplotlib inline 
import numpy as np
#关闭急切执行模式
tf.compat.v1.disable_eager_execution()
 
from tensorflow.examples.tutorials.mnist import input_data 
mnist = input_data.read_data_sets("F:\ANN\MNIST_data",one_hot = True)
#mnist = tf.keras.datasets.mnist 

Extracting F:\ANN\MNIST_data\train-images-idx3-ubyte.gz
Extracting F:\ANN\MNIST_data\train-labels-idx1-ubyte.gz
Extracting F:\ANN\MNIST_data\t10k-images-idx3-ubyte.gz
Extracting F:\ANN\MNIST_data\t10k-labels-idx1-ubyte.gz


In [3]:
#占位符
x = tf.placeholder(tf.float32,[None,784],name="X")  #每张图共有28*28=784个像素点
y = tf.placeholder(tf.float32,[None,10],name="Y")  #10个标签数据

In [ ]:
#输入层-隐藏层权重
H1_NN = 256  #隐藏层神经元数量
W1 = tf.Variable(tf.random_normal([784,H1_NN])) #正态分布随机数
B1 = tf.Variable(tf.zeros([H1_NN])) 

#隐藏层-输出层权重
W2 = tf.Variable(tf.random_normal([H1_NN,10])) #正态分布随机数
B2 = tf.Variable(tf.zeros([10])) 


In [ ]:
#计算
#输入-隐藏层
Y1 = tf.nn.relu(tf.matmul(x,W1)+B1)  #用到了relu激活函数

#隐藏-输出层
forward = tf.matmul(Y1,W2)+B2
pred = tf.nn.softmax(forward)

In [ ]:
#交叉熵损失函数
#平均交叉熵，该方法由于取对数容易导致在0附近数值不稳定
#loss_function = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1)）
                            

#新的损失函数定义方法,结合了softmax的交叉熵函数
loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=forward,labels=y))

In [ ]:
#设置训练超参数
train_epochs = 100 #超参数，训练次数
learning_rate   = 0.01  #学习率
batch_size = 50 #单次训练次数
total_batch = int(mnist.train.num_examples/batch_size) #训练数据总批数
display_step =2 #显示粒度

#定义梯度下降优化器，采用Adam优化法
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss_function)
#随机梯度下降优化SGD
#optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_function)

#定义准确率
#pred应该型如【0.01,0.8,0.07,0.02,0.03,0.02,0.01，0.02，0.01,0.01】这种按概率分布的值，通过argmax转换为【0:9】对应的值
correct_pretiction = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))  #相等为真，不相等为假
accuracy = tf.reduce_mean(tf.cast(correct_pretiction,tf.float32))    #布尔值转换为浮点数，并计算平均值

In [ ]:
from time import time
start_time = time() #记录开始时间
#创建会话，开始运行
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
acc_list = [] #记录每轮准确率
loss_list = []
#开始训练

for epoch in range(train_epochs):# 每一轮
    for batch in range(total_batch):  #按批次训练
        xs,ys = mnist.train.next_batch(batch_size)  #按批次读取数据
        _ = sess.run(optimizer,feed_dict={x:xs,y:ys})  #填充占位符变量
        
    #每轮训练完后利用验证集验证
    loss,acc = sess.run([loss_function,accuracy],
                       feed_dict= {x:mnist.validation.images,y:mnist.validation.labels})
    #准确率添加到列表
    acc_list.append(acc)
    loss_list.append(loss)
    if (epoch+1)%display_step == 0:
        print("epochs : ","%02d" % (epoch+1),\
              "loss=:","{:.9f}".format(loss),\
             "accury=:","{:.4f}".format(acc))
        
#画图可视化
#import matplotlib.pyplot as plt #载入matplot库
duration = time()-start_time #训练时间，显示
#plt.plot(acc_list,"r+")
plt.plot(acc_list)
plt.plot(loss_list)
print("train finished!","{:.4f}".format(duration))

In [ ]:
#测试集
acu_test = sess.run(accuracy,feed_dict= {x:mnist.test.images,y:mnist.test.labels})
print("Test accuracy:",acu_test)

In [ ]:
#找出错误预测
prediction_result = sess.run(tf.argmax(pred,1), feed_dict= {x:mnist.test.images})
compare_list = prediction_result==np.argmax(mnist.test.labels,1)
#print(compare_list)
err_list = [i for i in range(len(compare_list)) if compare_list[i]==False]
print(err_list,len(err_list))
